# Notebook to make Fig. 1

In [0]:
# Import necessary packages & set-up plotting aesthetics

import numpy as np 
import pylab
import pandas as pd
import lal
from scipy.stats import gaussian_kde
import importlib

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.lines import Line2D

import seaborn as sns
cp = sns.color_palette('muted')
cp2 = sns.color_palette('pastel')

import sys
sys.path.append('../scripts')
from helper_functions import *
sys.path.append('../utils')
import reconstructwf as rwf

style.use('plotting.mplstyle')

data_dir = '../data/' 

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


### Load in data: posteriors

Load in posterior and prior samples

In [1]:
td_samples = load_posterior_samples(data_dir)[0]

Generate KDE for $\chi_\mathrm{p}$ from the full run and the prior, for comparison 

In [2]:
# kde for full run 
samples_full = td_samples['full']
m1_full, m2_full = m1m2_from_mtotq(samples_full['mtotal'], samples_full['q'])
chip_full_samples = chi_precessing(m1_full, samples_full['chi1'], samples_full['tilt1'],
                          m2_full, samples_full['chi2'], samples_full['tilt2'])

chip_grid, chip_full_kde_on_grid = reflected_kde(chip_full_samples, 0, 1)

# kde for prior 
samples_prior = td_samples['prior']
m1_prior, m2_prior = m1m2_from_mtotq(samples_prior['mtotal'], samples_prior['q'])
chip_prior_samples = chi_precessing(m1_prior, samples_prior['chi1'], samples_prior['tilt1'],
                          m2_prior, samples_prior['chi2'], samples_prior['tilt2'])

_, chip_prior_kde_on_grid = reflected_kde(chip_prior_samples, 0, 1)

### Load in data: strain timeseries

Load in time-domain data from the detectors

In [3]:
data_dict =  np.load(data_dir+'LVC_strain_data.npy', allow_pickle=True).item()
wh_data_dict =  np.load(data_dir+'LVC_strain_data_whitened.npy', allow_pickle=True).item()
time_dict_data =  np.load(data_dir+'LVC_time_data.npy', allow_pickle=True).item()

Load in reconstructions from before/after each cutoff time

In [4]:
reconstruction_dict_all = np.load(data_dir+"waveform_reconstructions_all_detectors.npy", allow_pickle=True).item()
reconstruction_dict = {k:reconstruction_dict_all[k]['L1'] for k in reconstruction_dict_all.keys()}

Load in the SNRs corresponding to the reconstructions

In [16]:
# Load
snr_dict = np.load(data_dir+"snrs.npy", allow_pickle=True).item()

# Format into how we want to label them on the figure
snr_labels = {}
for run, snrs in snr_dict['L1'].items(): 

    # Calculate network SNR array
    network_snrs = np.asarray([
        np.sqrt(sum([snr**2 for snr in [L, H, V]])) for L, H, V in zip(snr_dict['L1'][run], 
                                                                       snr_dict['H1'][run], 
                                                                       snr_dict['V1'][run])
    ])
    
    # Calculate 5%, 50%, and 95% confidence intervals
    lower_CI, median, upper_CI = np.quantile(network_snrs[~np.isnan(network_snrs)], (0.05, 0.5, 0.95))
    
    # Get errorbars
    upper_ebar = upper_CI - median
    lower_ebar = median - lower_CI
    
    # Format
    snr_str = f"{median:.1f}^{{+{upper_ebar:.1f}}}_{{-{lower_ebar:.1f}}}"
    snr_labels[run] = snr_str

Define cutoff times 

In [12]:
# 0M <-> seconds
t0_0M_dict = {}
t0_0M_geo = 1242442967.405764
dt_10M = 0.0127 # 10 M = 12.7 ms 
dt_1M = dt_10M/10.

tstart = 1242442966.9077148
tend = 1242442967.607715
ra = 6.07546535866838
dec = -0.8000357325337637

# define t_0M in each detector in seconds
for ifo in ['H1', 'L1', 'V1']: 
    
    t_delay = lal.TimeDelayFromEarthCenter(lal.cached_detector_by_prefix[ifo].location, ra, dec, t0_0M_geo)
    t0_0M_dict[ifo] = t0_0M_geo + t_delay
    
# Max p strain from full run
lnP = reconstruction_dict['full']['params']['ln_posterior']
i_maxP = np.argmax(lnP)
L1_maxP_strain = np.asarray(reconstruction_dict['full']['h'][i_maxP])
L1_maxP_strain_wh = np.asarray(reconstruction_dict['full']['wh'][i_maxP])
L1_maxP_strain_bp = np.asarray(reconstruction_dict['full']['bp'][i_maxP])

# L1 strain
L1_LVC_strain = data_dict['L1']
L1_LVC_strain_wh = wh_data_dict['L1']

# Transform timeseries into into units of M
reconstruction_times_M = (reconstruction_dict['time samples']- t0_0M_dict['L1'])/dt_1M
L1_times_M = (time_dict_data['L1'] - t0_0M_dict['L1'])/dt_1M

Print some maxP parameters from full signal analysis:

In [13]:
maxP_params = reconstruction_dict['full']['params'][i_maxP]

m1_maxP, m2_maxP = m1m2_from_mtotq(maxP_params['mtotal'], maxP_params['q'])
chip_maxP = chi_precessing(m1_maxP, maxP_params['chi1'], maxP_params['tilt1'], 
                           m2_maxP, maxP_params['chi2'], maxP_params['tilt2'])

print('max P chi_p = ', chip_maxP)
print('max P mtotal = ', maxP_params['mtotal'])
print('max P q = ',  maxP_params['q'])

max P chi_p =  0.6242038304675497
max P mtotal =  267.1669838180254
max P q =  0.8892538113223768


### Make figure

In [ ]:
# Define times to plot
tc_to_plot = ['m40M', 'm20M', 'm10M', '20M', '30M']
n_to_plot = len(tc_to_plot)

# Cast strings to floats
tc_floats = []
for tc_str in tc_to_plot:
    if tc_str[0]=='m':
        tc_str_trimmed = tc_str[1:-1]
    else:
        tc_str_trimmed = tc_str[0:-1]     
    tc_float = -1*float(tc_str_trimmed) if tc_str[0]=='m' else float(tc_str_trimmed) 
    tc_floats.append(tc_float)

# Make figure 
fig, axes = plt.subplots(n_to_plot, 3, figsize=(15/1.3, 25*n_to_plot/8/1.3))

for ax in axes:
    for a in ax:
        a.set_rasterization_zorder(2)

for j,tc in enumerate(tc_floats):  
    
    # String for labeling the time step
    tc_str = tc_to_plot[j]
    lbl = tc_str.replace('m', '-') if tc_str[0]=='m' else tc_str
    lbl = lbl.replace('M', '\,M_\mathrm{ref}')
    
    # Middle plots: bandpassed waveforms with cutoff
    x_lims = [-100, 70]
    
    for k in [1,2]:
        if j==0:
            top_ax = axes[j][k].twiny()
            top_ax.plot(L1_times_M*dt_1M, L1_LVC_strain/(1e-22), color='k', alpha=0, zorder=3)
            top_ax.set_xlabel(r'$t~[s] $', fontsize=16, labelpad=10)
            top_ax.set_xlim(x_lims[0]*dt_1M, x_lims[1]*dt_1M)
            top_ax.grid(visible=False)
            axes[j][k].patch.set_visible(False)

        else:
            axes[j][k].plot(L1_times_M, L1_LVC_strain/(1e-22), color='k', alpha=0, zorder=3)
            
        axes[j][k].plot(reconstruction_times_M, L1_maxP_strain/(1e-22), color='k')
        axes[j][k].axvline(tc, ls='--', color='k', zorder=4)

        if j!=n_to_plot-1: 
            axes[j][k].set_xticklabels([])
        else: 
            axes[j][k].set_xlabel(r'$t~[M_\mathrm{ref}]$', fontsize=16)

        # Plot reconstructions before/after the cutoff
        n_reconstruction_to_plot = 300
        alpha_reconstruct = 0.02
        shading_kws = dict(hatch='/',  alpha=0.3, color='gray')
        if k==1:
            try:
                reconstructions = np.asarray(reconstruction_dict[f'insp {tc_str}']['h'])
            except: 
                reconstructions = np.asarray(reconstruction_dict['prior']['h'])
                
            idxs_to_plot = np.random.choice(reconstructions.shape[0], size=n_reconstruction_to_plot)
            color = cp[0]
            axes[j][k].plot(reconstruction_times_M, np.transpose(reconstructions[idxs_to_plot,:])/(1e-22), 
                            color=color, alpha=alpha_reconstruct, zorder=1)
            
            axes[j][k].axvspan(tc, x_lims[-1], **shading_kws, zorder=0)
            
            snr_lbl = snr_labels[f'insp {tc_str}']

        else:
            reconstructions = np.asarray(reconstruction_dict[f'rd {tc_str}']['h'])
            
            idxs_to_plot = np.random.choice(reconstructions.shape[0], size=n_reconstruction_to_plot)
            color = cp2[1]
            axes[j][k].plot(reconstruction_times_M, np.transpose(reconstructions[idxs_to_plot,:])/(1e-22), 
                            color=color, alpha=alpha_reconstruct, zorder=1)
            
            axes[j][k].axvspan(x_lims[0], tc, **shading_kws, zorder=0)
            
            snr_lbl = snr_labels[f'rd {tc_str}']

        axes[j][k].grid(color='silver', ls=':', alpha=0.7)
        axes[j][k].set_xlim(*x_lims)
        axes[j][k].set_ylim(-7, 7)
        
        # Add label for cutoff time
        textloc = tc + 4 if tc < 25 else tc - 43
        axes[j][k].text(textloc, 5, f'${lbl}$', color='k', fontsize=15, zorder=5)
        
        # Add label for SNR 
        axes[j][k].text(-92, -5.7, f'$\mathrm{{SNR}} = {snr_lbl}$', color='k', fontsize=13, zorder=5, 
                       bbox=dict(facecolor='white', edgecolor=color, boxstyle='round', alpha=0.9))

    axes[j][2].yaxis.set_label_position("right")
    axes[j][2].yaxis.tick_right()
    axes[j][2].set_ylabel(r'$h/10^{-22}$', fontsize=16)
    axes[j][1].set_yticklabels([])
    
    
    dx = -0.05
    x0, y0, x1, y1 = axes[j][1].get_position().bounds
    axes[j][1].set_position([x0+dx+0.0375, y0, x1, y1])
    x0, y0, x1, y1 = axes[j][2].get_position().bounds
    axes[j][2].set_position([x0+dx, y0, x1, y1])
    
    # Righthand plots: chi_p posteriors
    try:
        samples_insp = td_samples[f'insp {tc_str}']
        m1_insp, m2_insp = m1m2_from_mtotq(samples_insp['mtotal'], samples_insp['q'])
        chip_insp = chi_precessing(m1_insp, samples_insp['chi1'], samples_insp['tilt1'], 
                                   m2_insp, samples_insp['chi2'], samples_insp['tilt2'])
    except: 
        samples_insp = td_samples['prior']
        mask = np.random.choice(len(samples_insp), size=len(td_samples[f'rd {tc_str}']))
        m1_insp, m2_insp = m1m2_from_mtotq(samples_insp['mtotal'][mask], samples_insp['q'][mask])
        chip_insp = chi_precessing(m1_insp, samples_insp['chi1'][mask], samples_insp['tilt1'][mask], 
                                   m2_insp, samples_insp['chi2'][mask], samples_insp['tilt2'][mask])
    
    samples_rd = td_samples[f'rd {tc_str}']
    m1_rd, m2_rd = m1m2_from_mtotq(samples_rd['mtotal'], samples_rd['q'])
    chip_rd = chi_precessing(m1_rd, samples_rd['chi1'], samples_rd['tilt1'],
                                m2_rd, samples_rd['chi2'], samples_rd['tilt2'])
    
    axes[j][0].hist(chip_insp, histtype='step', bins=np.linspace(0,1,30), lw=1.5, 
                    color=cp[0], density=True)
    axes[j][0].hist(chip_rd, histtype='step', bins=np.linspace(0,1,30), lw=1.5, 
                    color=cp2[1], density=True)
    
    axes[j][0].plot(chip_grid, chip_full_kde_on_grid, color='k')
    prior_kws = dict(color='dimgray', ls=':')
    axes[j][0].plot(chip_grid, chip_prior_kde_on_grid, **prior_kws, lw=2)
    
    axes[j][0].set_ylabel(r'$p(\chi_\mathrm{p})$', fontsize=16)
    if j!=n_to_plot-1: 
        axes[j][0].set_xticklabels([])
    else: 
        axes[j][0].set_xlabel(r'$\chi_\mathrm{p}$', fontsize=16)
        axes[j][0].set_xticklabels(['0', '0.25', '0.5', '0.75', '1'])
    axes[j][0].set_xlim(0,1)
    axes[j][0].set_ylim(0,2.6)
    axes[j][0].grid(color='silver', ls=':', alpha=0.7)

    
    # Inset axis in leftmost column with whitened strain
    l = 0.4
    w = 0.3
    dx = 0.04
    axin = axes[j][0].inset_axes([dx-0.01, 1-dx-w, l, w])
    axin.plot(L1_times_M, L1_LVC_strain_wh, color='silver', lw=0.5)
    axin.plot(reconstruction_times_M, L1_maxP_strain_wh, color='k', lw=0.75)
    axin.axvline(tc, ls='--', color='k')
    axin.axvspan(tc, x_lims[-1], color=cp2[1], alpha=0.3, zorder=0)
    axin.axvspan(x_lims[0], tc, color=cp[0], alpha=0.3, zorder=0)
    axin.set_xlim(-60, 70)
    axin.set_ylim(-3.5, 3)
    axin.set_xticklabels([])
    axin.set_yticklabels([])
    
for ax in axes: 
    for a in ax:
        x0, y0, x1, y1 = a.get_position().bounds
        a.set_position([x0, y0, x1, y1+0.05/n_to_plot])

# Legend
handles = [
    Line2D([], [], color=cp[0], label='pre-cutoff analysis'),
    Line2D([], [], color=cp2[1], label='post-cutoff analysis'),
    Line2D([], [], color='k', label='full analysis'),
    Line2D([], [], **prior_kws, label='prior')
]
leg = axes[0][2].legend(
    handles=handles, 
    bbox_to_anchor=(-2.15, 1.4, 3.2, .102), loc='lower left', ncols=4, 
    mode="expand", borderaxespad=0., frameon=False, fontsize=14, handlelength=3
)
for i, h in enumerate(handles):
    leg.get_lines()[i].set_linewidth(3)

plt.savefig('figure_01.pdf', bbox_inches='tight')
plt.show()

/var/folders/w7/r9xkmg4d7n18hlq715yknq9w0000gq/T/ipykernel_4954/802020596.py:139: UserWarning: FixedFormatter should only be used together with FixedLocator
  axes[j][0].set_xticklabels(['0', '0.25', '0.5', '0.75', '1'])
